# TP2 : Extraction d'informations

Sur la base des éléments méthodologiques et des enseignements techniques présentés lors du cours théorique, il est demandé dans le cadre de ce TP :
- de se familiariser avec les techniques d’enrichissement (extraction de mots-clés, reconnaissance d’entités nommées, analyse de sentiment...)
- d’appliquer celles-ci au corpus de bulletins de la ville de Bruxelles préalablement convertis au format .txt

Pour ce faire, vous utiliserez différentes librairies Python vues au cours.

## 1. Keywords

In [ ]:
import os
import yake

In [ ]:
# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

In [ ]:
print(keywords)

## 2. Named Entity Recognition